In [1]:
import nibabel as nib
from neuromaps.datasets import available_annotations
from neuromaps import datasets
from neuromaps.resampling import resample_images
from neuromaps import parcellate
from nilearn.datasets import fetch_atlas_surf_destrieux
from neuromaps import datasets, images, nulls, resampling, plotting
from neuromaps.datasets import fetch_annotation
from neuromaps import transforms
from neuromaps.stats import compare_images
from brainspace.plotting import plot_surf
import numpy as np
from nibabel import freesurfer
from nibabel import gifti
import pandas as pd
from neuromaps.parcellate import Parcellater
from neuromaps import nulls

/dagher/dagher12/filip/software/miniconda3py/envs/py3/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


# UKBB

In [74]:
BMI_DKT_LH = scipy.io.loadmat('/dagher/dagher11/filip/Obesity_maps/data/UKBB_CT_DKT_LH.mat')['UKBB_CT_DKT_LH']
BMI_DKT_RH = scipy.io.loadmat('/dagher/dagher11/filip/Obesity_maps/data/UKBB_CT_DKT_RH.mat')['UKBB_CT_DKT_RH']
BMI_DKT=np.hstack((BMI_DKT_LH,BMI_DKT_RH))

In [75]:
DKT_gii_lh=images.annot_to_gifti('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT.annot')
DKT_gii_rh=images.annot_to_gifti('/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT.annot')

In [76]:
parc = Parcellater('/dagher/dagher11/filip/Obesity_maps/data_old/DKT_orig.nii.gz', 'mni152')  # DKT parcellater
parcL = Parcellater((DKT_gii_lh), 'fsaverage', hemi='L')  # DKT parcellater
parcR = Parcellater((DKT_gii_rh), 'fsaverage', hemi='R')  # DKT parcellater

results_df = pd.DataFrame(columns=['source', 'desc', 'corr', 'p'])

for annotation in available_annotations():
    try:
        print(annotation) 
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        if annotation[2]=='fsLR':
            brain_map=transforms.fslr_to_fsaverage(brain_map, target_density='164k')
        elif annotation[2]=='MNI152':
            brain_map=transforms.mni152_to_fsaverage(brain_map, fsavg_density='164k')
        NT_parc_lh = parcL.fit_transform(brain_map[0], 'fsaverage', hemi='L')
        NT_parc_rh = parcR.fit_transform(brain_map[1], 'fsaverage', hemi='R')
        NT_parc=np.concatenate((NT_parc_lh,NT_parc_rh))
        rotated = nulls.alexander_bloch(BMI_DKT, atlas='fsaverage', 
                                        density='164k', n_perm=10000, seed=1234,
                                        parcellation=list(('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT_new.gii',
                          '/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT_new.gii')))
        corr = compare_images(BMI_DKT, NT_parc, metric='pearsonr', nulls=rotated)
        print(corr)
        # Append the result to the dataframe
        results_df = results_df.append({'source': annotation[0], 'desc': annotation[1], 
                                        'corr': corr[0], 'p': corr[1]}, ignore_index=True)
        
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
(0.7721834838445021, 0.0004999500049995)
('aghourian2017', 'feobv', 'MNI152', '1mm')


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(-0.43472498258495185, 0.1044895510448955)
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.34000651286943867, 0.16978302169783022)
('bedard2019', 'feobv', 'MNI152', '1mm')


...

/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.5101288453056079, 0.004899510048995101)
('xu2020', 'evoexp', 'fsLR', '32k')


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(-0.0018196594826364084, 0.9909009099090091)


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


In [77]:
results_df.to_csv('/dagher/dagher11/filip/Obesity_maps/data/UKBB_corrs.csv', index=False)

# HCP Aging

In [27]:
BMI_DKT_LH=pd.read_csv('/dagher/dagher11/filip/Obesity_maps/data/HCPA_CT_DKT_LH.csv', header=None)
BMI_DKT_RH=pd.read_csv('/dagher/dagher11/filip/Obesity_maps/data/HCPA_CT_DKT_RH.csv', header=None)
BMI_DKT=pd.concat((BMI_DKT_LH,BMI_DKT_RH))

In [28]:
DKT_gii_lh=images.annot_to_gifti('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT.annot')
DKT_gii_rh=images.annot_to_gifti('/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT.annot')

In [29]:
parc = Parcellater('/dagher/dagher11/filip/Obesity_maps/data_old/DKT_orig.nii.gz', 'mni152')  # DKT parcellater
parcL = Parcellater((DKT_gii_lh), 'fsaverage', hemi='L')  # DKT parcellater
parcR = Parcellater((DKT_gii_rh), 'fsaverage', hemi='R')  # DKT parcellater

results_df = pd.DataFrame(columns=['source', 'desc', 'corr', 'p'])

for annotation in available_annotations():
    try:
        print(annotation) 
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        if annotation[2]=='fsLR':
            brain_map=transforms.fslr_to_fsaverage(brain_map, target_density='164k')
        elif annotation[2]=='MNI152':
            brain_map=transforms.mni152_to_fsaverage(brain_map, fsavg_density='164k')
        NT_parc_lh = parcL.fit_transform(brain_map[0], 'fsaverage', hemi='L')
        NT_parc_rh = parcR.fit_transform(brain_map[1], 'fsaverage', hemi='R')
        NT_parc=np.concatenate((NT_parc_lh,NT_parc_rh))
        rotated = nulls.alexander_bloch(BMI_DKT.values, atlas='fsaverage', 
                                        density='164k', n_perm=10000, seed=1234,
                                        parcellation=list(('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT_new.gii',
                          '/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT_new.gii')))
        corr = compare_images(BMI_DKT.values, NT_parc, metric='pearsonr', nulls=rotated)
        print(corr)
        # Append the result to the dataframe
        results_df = results_df.append({'source': annotation[0], 'desc': annotation[1], 
                                        'corr': corr[0], 'p': corr[1]}, ignore_index=True)
        
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
(0.11145602187247225, 0.5603439656034397)
('aghourian2017', 'feobv', 'MNI152', '1mm')


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.052567848065293146, 0.7628237176282372)
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.11735015816896394, 0.47975202479752027)
('bedard2019', 'feobv', 'MNI152', '1mm')


...

/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.013669289958613303, 0.9285071492850715)
('xu2020', 'evoexp', 'fsLR', '32k')


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.05962294015623706, 0.663033696630337)


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


In [30]:
results_df.to_csv('/dagher/dagher11/filip/Obesity_maps/data/HCPA_corrs.csv', index=False)

# ABCD

In [78]:
BMI_DKT_LH = scipy.io.loadmat('/dagher/dagher11/filip/Obesity_maps/data/ABCD_CT_DKT_LH.mat')['ABCD_CT_DKT_LH']
BMI_DKT_RH = scipy.io.loadmat('/dagher/dagher11/filip/Obesity_maps/data/ABCD_CT_DKT_RH.mat')['ABCD_CT_DKT_RH']
BMI_DKT=np.hstack((BMI_DKT_LH,BMI_DKT_RH))

In [79]:
parc = Parcellater('/dagher/dagher11/filip/Obesity_maps/data_old/DKT_orig.nii.gz', 'mni152')  # DKT parcellater
parcL = Parcellater((DKT_gii_lh), 'fsaverage', hemi='L')  # DKT parcellater
parcR = Parcellater((DKT_gii_rh), 'fsaverage', hemi='R')  # DKT parcellater

results_df = pd.DataFrame(columns=['source', 'desc', 'corr', 'p'])

for annotation in available_annotations():
    try:
        print(annotation) 
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        if annotation[2]=='fsLR':
            brain_map=transforms.fslr_to_fsaverage(brain_map, target_density='164k')
        elif annotation[2]=='MNI152':
            brain_map=transforms.mni152_to_fsaverage(brain_map, fsavg_density='164k')
        NT_parc_lh = parcL.fit_transform(brain_map[0], 'fsaverage', hemi='L')
        NT_parc_rh = parcR.fit_transform(brain_map[1], 'fsaverage', hemi='R')
        NT_parc=np.concatenate((NT_parc_lh,NT_parc_rh))
        rotated = nulls.alexander_bloch(BMI_DKT, atlas='fsaverage', 
                                        density='164k', n_perm=10000, seed=1234,
                                        parcellation=list(('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT_new.gii',
                          '/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT_new.gii')))
        corr = compare_images(BMI_DKT, NT_parc, metric='pearsonr', nulls=rotated)
        print(corr)
        # Append the result to the dataframe
        results_df = results_df.append({'source': annotation[0], 'desc': annotation[1], 
                                        'corr': corr[0], 'p': corr[1]}, ignore_index=True)
        
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
(0.3406333354987286, 0.16598340165983402)
('aghourian2017', 'feobv', 'MNI152', '1mm')


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.01805016075327069, 0.9333066693330667)
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.22721182004988916, 0.28887111288871115)
('bedard2019', 'feobv', 'MNI152', '1mm')


...

/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.29396411640574277, 0.11778822117788221)
('xu2020', 'evoexp', 'fsLR', '32k')


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(-0.1905804367392052, 0.2328767123287671)


/tmp/ipykernel_32031/2019287863.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


In [80]:
results_df.to_csv('/dagher/dagher11/filip/Obesity_maps/data/ABCD_corrs.csv', index=False)

# HCP

In [8]:
BMI_DKT_LH=pd.read_csv('/dagher/dagher11/filip/Obesity_maps/data/HCP_CT_DKT_LH.csv', header=None)
BMI_DKT_RH=pd.read_csv('/dagher/dagher11/filip/Obesity_maps/data/HCP_CT_DKT_RH.csv', header=None)
BMI_DKT=pd.concat((BMI_DKT_LH,BMI_DKT_RH))

In [3]:
DKT_gii_lh=images.annot_to_gifti('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT.annot')
DKT_gii_rh=images.annot_to_gifti('/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT.annot')

In [16]:
parc = Parcellater('/dagher/dagher11/filip/Obesity_maps/data_old/DKT_orig.nii.gz', 'mni152')  # DKT parcellater
parcL = Parcellater((DKT_gii_lh), 'fsaverage', hemi='L')  # DKT parcellater
parcR = Parcellater((DKT_gii_rh), 'fsaverage', hemi='R')  # DKT parcellater

results_df = pd.DataFrame(columns=['source', 'desc', 'corr', 'p'])

for annotation in available_annotations():
    try:
        print(annotation) 
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        if annotation[2]=='fsLR':
            brain_map=transforms.fslr_to_fsaverage(brain_map, target_density='164k')
        elif annotation[2]=='MNI152':
            brain_map=transforms.mni152_to_fsaverage(brain_map, fsavg_density='164k')
        NT_parc_lh = parcL.fit_transform(brain_map[0], 'fsaverage', hemi='L')
        NT_parc_rh = parcR.fit_transform(brain_map[1], 'fsaverage', hemi='R')
        NT_parc=np.concatenate((NT_parc_lh,NT_parc_rh))
        rotated = nulls.alexander_bloch(BMI_DKT.values, atlas='fsaverage', 
                                        density='164k', n_perm=10000, seed=1234,
                                        parcellation=list(('/dagher/dagher11/filip/Obesity_maps/data_old/lh.DKT_new.gii',
                          '/dagher/dagher11/filip/Obesity_maps/data_old/rh.DKT_new.gii')))
        corr = compare_images(BMI_DKT.values, NT_parc, metric='pearsonr', nulls=rotated)
        print(corr)
        # Append the result to the dataframe
        results_df = results_df.append({'source': annotation[0], 'desc': annotation[1], 
                                        'corr': corr[0], 'p': corr[1]}, ignore_index=True)
        
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
(-0.2662699090568067, 0.07739226077392261)
('aghourian2017', 'feobv', 'MNI152', '1mm')


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.05201223196426037, 0.7135286471352865)
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.035125672366532544, 0.7818218178182181)
('bedard2019', 'feobv', 'MNI152', '1mm')


...

/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(-0.25621297048244124, 0.07169283071692831)
('xu2020', 'evoexp', 'fsLR', '32k')


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


(0.19897622896272787, 0.11528847115288471)


/tmp/ipykernel_1818/1424287289.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'source': annotation[0], 'desc': annotation[1],


In [18]:
results_df.to_csv('/dagher/dagher11/filip/Obesity_maps/data/HCP_corrs.csv', index=False)

# OLD

## Read the basic files

In [2]:
BMI_HCP=list(['/dagher/dagher11/filip/Obesity_maps/data_old/BMI_lh.gii',
          '/dagher/dagher11/filip/Obesity_maps/data_old/BMI_rh.gii'])

In [3]:
BMI_ctx=transforms.fsaverage_to_fsaverage(BMI_HCP, '164k')

## Read mask of medial wall for fsaverage

In [4]:
medwall_idx_fsaverage_lh=(freesurfer.io.read_label('/dagher/dagher10/yyau/toolbox/freesurfer/subjects/fsaverage/label/lh.Medial_wall.label',
                        read_scalars=False))
medwall_idx_fsaverage_rh=(freesurfer.io.read_label('/dagher/dagher10/yyau/toolbox/freesurfer/subjects/fsaverage/label/rh.Medial_wall.label',
                        read_scalars=False))

## Set all vertices of medial wall to nan

In [5]:
BMI_ctx[0].agg_data()[medwall_idx_fsaverage_lh] = np.nan
BMI_ctx[1].agg_data()[medwall_idx_fsaverage_rh] = np.nan

## Transform fsaverage 164k to 10k for further analyses - saves computational power

In [9]:
BMI_10k=transforms.fsaverage_to_fsaverage(BMI_ctx, '10k')

## Save files for plotting in the future

In [16]:
nib.save(BMI_10k[1], '/dagher/dagher11/filip/Obesity_maps/data/BMI_10k_R.gii')
nib.save(BMI_10k[0], '/dagher/dagher11/filip/Obesity_maps/data/BMI_10k_L.gii')


## Save annotations transformed to 10k space

In [10]:
for annotation in available_annotations():
    try:
        print(annotation)
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        BMI_res, brain_map_res = resample_images(src=BMI_HCP, trg=brain_map, src_space='fsaverage', 
                                                 trg_space=annotation[2], method='linear', 
                                                 resampling='transform_to_src')
        #rotated = nulls.alexander_bloch(BMI_res, atlas='fsaverage', density='10k', n_perm=10000, seed=1234)
        nib.save(brain_map_res[0], '/dagher/dagher11/filip/Obesity_maps/data_old/'+annotation[0]+annotation[1]+'_L.gii')
        nib.save(brain_map_res[1], '/dagher/dagher11/filip/Obesity_maps/data_old/'+annotation[0]+annotation[1]+'_R.gii')
        #corr = compare_images(BMI_res, brain_map_res, metric='pearsonr', nulls=rotated)
        #print(corr)
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
('aghourian2017', 'feobv', 'MNI152', '1mm')
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')
('bedard2019', 'feobv', 'MNI152', '1mm')
('beliveau2017', 'az10419369', 'MNI152', '1mm')
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
('beliveau2017', 'cumi101', 'MNI152', '1mm')
('beliveau2017', 'dasb', 'MNI152', '1mm')
('beliveau2017', 'sb207145', 'MNI152', '1mm')
('ding2010', 'mrb', 'MNI152', '1mm')
('dubois2015', 'abp688', 'MNI152', '1mm')
('dukart2018', 'flumazenil', 'MNI152', '3mm')
('dukart2018', 'fpcit', 'MNI152', '3mm')
('fazio2016', 'madam', 'MNI152', '3mm')
('finnema2016', 'ucbj', 'MNI152', '1mm')
('gallezot2010', 'p943', 'MNI152', '1mm')
('gallezot2017', 'gsk189254', 'MNI152', '1mm')
('hcps1200', 'megalpha', 'fsLR', '4k')
('hcps1200', 'megbeta', 'fsLR', '4k')
('hcps1200', 'megdelta', 'fsLR', '4k')
('hcps1200', 'meggamma1', 'fsLR', '4k')
('hcps1200', 'meggamma2', 'fsLR', '4k')
('hcps1200', 'megtheta', 'fsLR', '4k')
('hcps1200', 'megtimescale

# Calculate correlations with all available annotations

In [430]:
for annotation in available_annotations():
    try:
        print(annotation)
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        BMI_res, brain_map_res = resample_images(src=BMI_10k, trg=brain_map, src_space='fsaverage', 
                                                 trg_space=annotation[2], method='linear', 
                                                 resampling='transform_to_src')
        rotated = nulls.alexander_bloch(BMI_res, atlas='fsaverage', density='10k', n_perm=10000, seed=1234)
        corr = compare_images(BMI_res, brain_map_res, metric='pearsonr', nulls=rotated)
        print(corr)
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
(0.14673678576946259, 0.5224477552244775)
('aghourian2017', 'feobv', 'MNI152', '1mm')
(-0.09255792200565338, 0.5381461853814619)
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')
(0.14988219738006592, 0.12498750124987501)
('bedard2019', 'feobv', 'MNI152', '1mm')
(-0.15804708003997803, 0.20957904209579042)
('beliveau2017', 'az10419369', 'MNI152', '1mm')
(0.012863333337008953, 0.793920607939206)
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
(-0.01622484065592289, 0.8091190880911909)
('beliveau2017', 'cumi101', 'MNI152', '1mm')
(-0.1589098423719406, 0.25047495250474955)
('beliveau2017', 'dasb', 'MNI152', '1mm')
(-0.23323631286621094, 0.011998800119988001)
('beliveau2017', 'sb207145', 'MNI152', '1mm')
(-0.055843595415353775, 0.57994200579942)
('ding2010', 'mrb', 'MNI152', '1mm')
(0.13911652565002441, 0.41235876412358763)
('dubois2015', 'abp688', 'MNI152', '1mm')
(-0.026094626635313034, 0.8054194580541946)
('dukart2018', 'flumazenil', 'MNI152', '3mm'

# Surface area

# Read the basic files

In [26]:
BMI_HCP_area=list(['/dagher/dagher11/filip/Obesity_maps/data/BMI_area_lh.gii',
          '/dagher/dagher11/filip/Obesity_maps/data/BMI_area_rh.gii'])

In [27]:
BMI_ctx_area=transforms.fsaverage_to_fsaverage(BMI_HCP_area, '164k')

# Set all vertices of medial wall to nan

In [28]:
BMI_ctx_area[0].agg_data()[medwall_idx_fsaverage_lh] = np.nan
BMI_ctx_area[1].agg_data()[medwall_idx_fsaverage_rh] = np.nan

# Transform fsaverage 164k to 10k for further analyses

In [29]:
BMI_10k_area=transforms.fsaverage_to_fsaverage(BMI_ctx_area, '10k')

# Save files for plotting in the future

In [30]:
nib.save(BMI_10k_area[1], '/dagher/dagher11/filip/Obesity_maps/data/BMI_area_10k_R.gii')
nib.save(BMI_10k_area[0], '/dagher/dagher11/filip/Obesity_maps/data/BMI_area_10k_L.gii')


# Calculate correlations with all available annotations

In [31]:
for annotation in available_annotations():
    try:
        print(annotation)
        brain_map=datasets.fetch_annotation(source=annotation[0], desc=annotation[1])
        BMI_res, brain_map_res = resample_images(src=BMI_10k_area, trg=brain_map, src_space='fsaverage', 
                                                 trg_space=annotation[2], method='linear', 
                                                 resampling='transform_to_src')
        rotated = nulls.alexander_bloch(BMI_res, atlas='fsaverage', density='10k', n_perm=10000, seed=1234)
        corr = compare_images(BMI_res, brain_map_res, metric='pearsonr', nulls=rotated)
        print(corr)
    except:
        pass

('abagen', 'genepc1', 'fsaverage', '10k')
(-0.19280381500720978, 0.35696430356964304)
('aghourian2017', 'feobv', 'MNI152', '1mm')
(0.16438035666942596, 0.24067593240675933)
('alarkurtti2015', 'raclopride', 'MNI152', '3mm')
(-0.05113248527050018, 0.6778322167783222)
('bedard2019', 'feobv', 'MNI152', '1mm')
(-0.01393503975123167, 0.9252074792520748)
('beliveau2017', 'az10419369', 'MNI152', '1mm')
(-0.07088306546211243, 0.12068793120687932)
('beliveau2017', 'cimbi36', 'MNI152', '1mm')
(-0.09123453497886658, 0.14878512148785122)
('beliveau2017', 'cumi101', 'MNI152', '1mm')
(-0.08148433268070221, 0.6078392160783922)
('beliveau2017', 'dasb', 'MNI152', '1mm')
(-0.13014158606529236, 0.26047395260473954)
('beliveau2017', 'sb207145', 'MNI152', '1mm')
(-0.08900177478790283, 0.33236676332366766)
('ding2010', 'mrb', 'MNI152', '1mm')
(0.056388936936855316, 0.7453254674532547)
('dubois2015', 'abp688', 'MNI152', '1mm')
(-0.03826126456260681, 0.7101289871012899)
('dukart2018', 'flumazenil', 'MNI152', '